# Week 08

1. Count number of trainable parameters in $\text{BERT}_{\text{LARGE}}$
2. Go through this [tutorial](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html) until *Set Anomaly Detection*
3. Read about [KV Caching](https://neptune.ai/blog/transformers-key-value-caching):
    - What is it?
    - Why is it useful?
    - How does it work?
    - Make the code in the tutorial run